> **Tried to analyse the spotify dataset and find out the relations between various conents of a song**

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
data=pd.read_csv('/kaggle/input/top50spotify2019/top50.csv',encoding="iso-8859-1",index_col=0)

In [ ]:
data.head()

In [ ]:
data.shape

In [ ]:
data.describe().T

In [ ]:
data.info()

In [ ]:
for i in data.columns:
    print('The number of unique ',i)
    print(data[i].nunique())

In [ ]:
print(data['Track.Name'].value_counts())

In [ ]:
data.isna().sum()

In [ ]:
from pylab import rcParams
rcParams['figure.figsize'] = 15,12
data.hist();

In [ ]:
num_df = data.select_dtypes(include=['int64','float64'])

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
fig = plt.figure(figsize=(35,50))
for i in range(0,len(num_df.columns)):
    ax=fig.add_subplot(14,2,i+1)
    sns.boxplot(num_df[num_df.columns[i]])

In [ ]:
sns.heatmap(num_df.corr(),annot=True);

* relation between dB and acous
* dB and nrgy
* Energy and Loudness are highly correlated, meanwhile the other variable associated to sound and energy like danceability is also correlated.
* Instrumentalness & accousticness is highly negative correlated to Loudness.
* A song depends on duration,year it was launched and we should focus to check what all makes it to the top of the list and find the features which are impactful in the list such as year and duration,year is a very good indicator .

In [ ]:
corr = pd.DataFrame(num_df.corr())
corr

In [ ]:
relational_impact = pd.DataFrame({'Impact values' : corr.sum()}).sort_values(by='Impact values',ascending=False)
relational_impact

Variables like Loudness, Energy, year and danceability are impacting many other variables. So these four variables can become a common features/tastes of the users. We can use probability functions on these variables to create customisations

In [ ]:
sns.scatterplot(data['Loudness..dB..'],data['Energy']);

In [ ]:
def normalize(column):
    upper = column.max()
    lower = column.min()
    y = (column - lower)/(upper-lower)
    return y

In [ ]:
for i in data.columns:
    if(data[i].dtype=='int64'or data[i].dtype=='float64'):
        data[i]=normalize(data[i])

In [ ]:
data.head()

In [ ]:
data['Popularity_Category'] = pd.cut(data['Popularity'],bins = (-1,0.30,0.8,1), labels =('Low','Medium','High'))
print(data['Popularity_Category'].value_counts())
print('\n')
print('Total count after new variable : ' ,data['Popularity_Category'].value_counts().sum())
print('\n')
print('Shape of the original data: ' ,data.shape)

In [ ]:
popularity_features = data.groupby(data['Popularity_Category']).mean()
popularity_features

In [ ]:
popularity_features[['Danceability','Popularity','Loudness..dB..','Length.','Liveness']].plot();

> Inferences from Popularity
> Liveness ,Dancebility and length have almost same kind of trend .

In [ ]:
sns.pairplot(data = num_df, corner=True)

In [ ]:
artist_wise = data.groupby(data['Artist.Name']).mean()
artist_wise.sort_values(by='Popularity',ascending=False).head(50).mean()

> Once again it is evident that the Loudness and Danceability are clear contributors for the Popularity. As we can see the top 50 popular artists' summary